In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from lyricsgenius import Genius
import pickle
import numpy as np

In [2]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
genius = Genius(API_KEY)
genius.remove_section_headers = True


In [3]:
df = pd.read_csv('Hot 100.csv')

In [4]:
df['chart_date'] = pd.to_datetime(df.chart_date)
df['chart_debut'] = pd.to_datetime(df.chart_debut)

In [5]:
df['consecutive_weeks'].fillna(value=0,inplace=True)
df['previous_week'].fillna(value=0,inplace=True)

In [6]:
frame = pd.DataFrame(df.dtypes)
numericals = list(frame.loc[frame.iloc[:,0]=='float64'].iloc[:,0].keys())
numericals = numericals + list(frame.loc[frame.iloc[:,0]=='int64'].iloc[:,0].keys())
for x in numericals:
    df[x] = df[x].astype('int32')

In [8]:
df.sort_values(by='chart_date',inplace=True)
df.to_pickle('Hot100')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335195 entries, 100546 to 79691
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   chart_position     335195 non-null  int32         
 1   chart_date         335195 non-null  datetime64[ns]
 2   song               335195 non-null  object        
 3   performer          335195 non-null  object        
 4   song_id            335195 non-null  object        
 5   instance           335195 non-null  int32         
 6   time_on_chart      335195 non-null  int32         
 7   consecutive_weeks  335195 non-null  int32         
 8   previous_week      335195 non-null  int32         
 9   peak_position      335195 non-null  int32         
 10  worst_position     335195 non-null  int32         
 11  chart_debut        335195 non-null  datetime64[ns]
 12  chart_url          335195 non-null  object        
dtypes: datetime64[ns](2), int32(7), object(4

In [9]:

unique_df = df.drop_duplicates(subset=['song_id'])

In [10]:
first_pass_df = pd.read_pickle('./complete_df.pkl')
missing_df = first_pass_df.loc[first_pass_df.lyrics.isna()]
missing_df = missing_df.merge(unique_df,how='left',on='song_id')
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9444 entries, 0 to 9443
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   chart_position_x     9444 non-null   int32         
 1   chart_date_x         9444 non-null   datetime64[ns]
 2   song_x               9444 non-null   object        
 3   performer_x          9444 non-null   object        
 4   song_id              9444 non-null   object        
 5   instance_x           9444 non-null   int32         
 6   time_on_chart_x      9444 non-null   int32         
 7   consecutive_weeks_x  9444 non-null   int32         
 8   previous_week_x      9444 non-null   int32         
 9   peak_position_x      9444 non-null   int32         
 10  worst_position_x     9444 non-null   int32         
 11  chart_debut_x        9444 non-null   datetime64[ns]
 12  chart_url_x          9444 non-null   object        
 13  lyrics               0 non-null  

In [11]:
unique_df.columns

Index(['chart_position', 'chart_date', 'song', 'performer', 'song_id',
       'instance', 'time_on_chart', 'consecutive_weeks', 'previous_week',
       'peak_position', 'worst_position', 'chart_debut', 'chart_url'],
      dtype='object')

In [12]:
fill_df = pd.DataFrame()

In [13]:
fill_df['chart_position'] = missing_df['chart_position_y']
fill_df['chart_date'] = missing_df['chart_date_y']
fill_df['song'] = missing_df['song_y']
fill_df['performer'] = missing_df['performer_y']
fill_df['song_id'] = missing_df['song_id']
fill_df['instance'] = missing_df['instance_y']
fill_df['time_on_chart'] = missing_df['time_on_chart_y']
fill_df['consecutive_weeks'] = missing_df['consecutive_weeks_y']
fill_df['previous_week'] = missing_df['previous_week_y']
fill_df['peak_position'] = missing_df['peak_position_y']
fill_df['worst_position'] = missing_df['worst_position_y']
fill_df['chart_debut'] = missing_df['chart_debut_y']
fill_df['chart_url'] = missing_df['chart_url_y']
fill_df['lyrics'] = missing_df['lyrics']
fill_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9444 entries, 0 to 9443
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_position     9444 non-null   int32         
 1   chart_date         9444 non-null   datetime64[ns]
 2   song               9444 non-null   object        
 3   performer          9444 non-null   object        
 4   song_id            9444 non-null   object        
 5   instance           9444 non-null   int32         
 6   time_on_chart      9444 non-null   int32         
 7   consecutive_weeks  9444 non-null   int32         
 8   previous_week      9444 non-null   int32         
 9   peak_position      9444 non-null   int32         
 10  worst_position     9444 non-null   int32         
 11  chart_debut        9444 non-null   datetime64[ns]
 12  chart_url          9444 non-null   object        
 13  lyrics             0 non-null      object        
dtypes: datet

In [14]:
def get_lyrics_genius(row):
    ans = float('nan')
    title = row['song']
    artist = row['performer']
    try:
        song = genius.search_song(title, artist=artist)
    except:
        ans = float('nan')
    try:
        song.lyrics
    except:
        ans = float('nan')
    else:
        if song.lyrics[:len(title)]==title:
            ans = song.lyrics[len(title)+len(' lyrics'):]
        else:
            ans = float('nan')
    return(ans)


In [15]:
fill_df['lyrics'] = fill_df.apply(get_lyrics_genius,axis=1)

Searching for "Betty Lou Got A New Pair Of Shoes" by Bobby Freeman...
Done.
Searching for "Nel Blu Dipinto Di Blu (Volaré)" by Domenico Modugno...
Done.
Searching for "All I Have To Do Is Dream" by The Everly Brothers...
Searching for "Guess Things Happen That Way" by Johnny Cash And The Tennessee Two...
Done.
Searching for "Let's Go Steady For The Summer" by The Three G's...
Done.
Searching for "Happy Years" by The Diamonds...
Done.
Searching for "The Freeze" by Tony And Joe...
Done.
Searching for "Patricia" by Perez Prado And His Orchestra...
No results found for: 'Patricia Perez Prado And His Orchestra'
Searching for "Got A Match?" by Frank Gallup...
Done.
Searching for "Got A Match?" by The Daddy-O's...
Done.
Searching for "Over And Over" by Thurston Harris...
Done.
Searching for "Op" by The Honeycones...
Done.
Searching for "Rebel-'rouser" by Duane Eddy His Twangy Guitar And The Rebels...
No results found for: 'Rebel-'rouser Duane Eddy His Twangy Guitar And The Rebels'
Searching f

In [19]:
done_df = fill_df.loc[fill_df.lyrics.notna()]
not_done_df = fill_df.loc[fill_df.lyrics.isna()]

In [20]:
done_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 354 to 9163
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_position     82 non-null     int32         
 1   chart_date         82 non-null     datetime64[ns]
 2   song               82 non-null     object        
 3   performer          82 non-null     object        
 4   song_id            82 non-null     object        
 5   instance           82 non-null     int32         
 6   time_on_chart      82 non-null     int32         
 7   consecutive_weeks  82 non-null     int32         
 8   previous_week      82 non-null     int32         
 9   peak_position      82 non-null     int32         
 10  worst_position     82 non-null     int32         
 11  chart_debut        82 non-null     datetime64[ns]
 12  chart_url          82 non-null     object        
 13  lyrics             82 non-null     object        
dtypes: datet

In [21]:
not_done_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9362 entries, 0 to 9443
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_position     9362 non-null   int32         
 1   chart_date         9362 non-null   datetime64[ns]
 2   song               9362 non-null   object        
 3   performer          9362 non-null   object        
 4   song_id            9362 non-null   object        
 5   instance           9362 non-null   int32         
 6   time_on_chart      9362 non-null   int32         
 7   consecutive_weeks  9362 non-null   int32         
 8   previous_week      9362 non-null   int32         
 9   peak_position      9362 non-null   int32         
 10  worst_position     9362 non-null   int32         
 11  chart_debut        9362 non-null   datetime64[ns]
 12  chart_url          9362 non-null   object        
 13  lyrics             0 non-null      object        
dtypes: datet

In [22]:
done_df.to_pickle('./2ndpassdataframe.pkl')

In [ ]:
not_done_df.to_pickle('./3rdpassdataframe.pkl')